##### Task 1 (8 Points)

In [23]:
"""
    topic: Paths in triangles

    a)
    Implement an algorithm that constructs such a triangle with a depth of d and stores it as a list of length n.
    choose numbers from 0, ..., 99 for the entries

    b) 
    Implement an algorithm with a runtime of O(n) that calculates the shortest path in a triangular structure. 
    In this context, the length of a path is determined by summing the values of all visited points in the triangle. 
    Movement options are limited to going up left or up right from a given point within the triangle.

"""
import numpy as np
def triangle(depth):
    number = np.arange(1, depth + 2)
    return np.random.randint(0,100,size=np.sum(number)), number
    
def min_path(triangle):
    pass

##### Task 2 (12 Points)

In [ ]:
""" 
    topic: graphs

    a)
    Implement and utilize a Graph class for constructing a graph. 
    The class should include functionality for adding edges both through the add_edge() method and from a file named graph.txt. 
    The file format is expected to have the number of vertices in the first row, followed by edge information below it.

    b) 
    Implement the number_of_neighbors() function within the Node class.

    c)
    Implement the is_simple() function  within Graph class.

    d) 
    Implement the adjancency_matrix() function within Graph class.
    Store the adjacency matrix as an ndarray.
"""


